In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import requests
from io import BytesIO

In [2]:
# Read the excel sheet and skip blank rows
raw_excel = pd.read_excel("data/DSHA LIHTC List_MAPPING.xlsx", engine='openpyxl', skiprows=[2,3], skipfooter=4, dtype=str)
# Add additional column information from the first row
raw_excel.columns = (raw_excel.columns.astype(str) + " " + raw_excel.head(1).fillna("").astype(str)).iloc[0].str.strip().values
raw_excel.rename(columns={"ALLOCATION .1 DATE": "ALLOCATION DATE", "ALLOCATION  AMOUNT": "ALLOCATION AMOUNT", "Type of Property*": "Type of Property"}, inplace=True)
raw_excel.drop(0, inplace=True)

In [3]:
# Find and label the rows about tax year
raw_excel["is tax"] = raw_excel["PROJECT NAME & ADDRESS"].str.contains("TAX CREDIT ALLOCATIONS")

# Function that maps each the boolean column "is tax", which is true when a row contains tax year information, to an integer equal to the tax year
def assign_to_year(x, i):
    # if the row is a tax year, increment i
    if x:
        i[0] = i[0] + 1
    # return an integer equal to the tax year
    return i[0] + 2016

# add a column for the tax year
index = [-1]
raw_excel["Tax Allocation Year"] = raw_excel["is tax"].apply(assign_to_year, args=[index])

# drop rows of tax year information and reformat
raw_excel = raw_excel.loc[~raw_excel["is tax"]].drop(columns="is tax").reset_index(drop=True)


In [4]:
# Add a column that labels the primary line for an entry
raw_excel["primary"] = ~raw_excel["County"].isna()

# Function that maps each the boolean column "primary", which is true when a row contains the primary info from the dataset, to an integer that functions as an index for primary entries
def assign_to_year(x, i):
    # if the row is primary, increment i
    if x:
        i[0] = i[0] + 1
    # return an index for the primary entries
    return i[0]

# add an index column for the primary entries
index = [-1]
raw_excel["primary"] = raw_excel["primary"].apply(assign_to_year, args=[index])

In [5]:
# Fix 'ALLOCATION AMOUNT', 'ALLOCATION DATE' swap
flipped_years = [2018, 2019, 2020, 2021, 2022]
#raw_excel["ALLOCATION AMOUNT"] = raw_excel["ALLOCATION AMOUNT"].astype(str)
#raw_excel['ALLOCATION DATE'] = raw_excel['ALLOCATION DATE'].astype(str)
tmp = raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years)]['ALLOCATION AMOUNT'].copy()
tmp2 = raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years)]['ALLOCATION DATE'].copy()
raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years), 'ALLOCATION AMOUNT'] = tmp2.values

raw_excel.loc[raw_excel["Tax Allocation Year"].isin(flipped_years), 'ALLOCATION DATE'] = tmp.values


In [6]:
# Create separate dataframes for each row in a data entry
grouped_data = raw_excel.groupby("primary")
raw_data1 = grouped_data.nth(0)
raw_data2 = grouped_data.nth(1).drop(columns="Tax Allocation Year")
raw_data3 = grouped_data.nth(2).drop(columns="Tax Allocation Year")
raw_data4 = grouped_data.nth(3).drop(columns="Tax Allocation Year")
raw_data5 = grouped_data.nth(4).drop(columns="Tax Allocation Year")

# Modify the column names for each dataframe to prepare for joining
raw_data2.columns = raw_data2.columns + " 2"
raw_data3.columns = raw_data3.columns + " 3"
raw_data4.columns = raw_data4.columns + " 4"
raw_data5.columns = raw_data5.columns + " 5"

# Join the dataframes by index
flattened_data = raw_data1.join(raw_data2, how="left").join(raw_data3, how="left").join(raw_data4, how="left").join(raw_data5, how="left").dropna(axis=1, how='all').reset_index(drop=True)


In [7]:
# Convert dates back to their orginial format
flattened_data["Placed in Service Date"] = pd.to_datetime(flattened_data["Placed in Service Date"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["ALLOCATION DATE"] = pd.to_datetime(flattened_data["ALLOCATION DATE"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Tax Credit Compliance Date"] = pd.to_datetime(flattened_data["Tax Credit Compliance Date"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Extended Use Period"] = pd.to_datetime(flattened_data["Extended Use Period"], errors='coerce').dt.strftime('%m/%d/%Y')
flattened_data["Placed in Service Date 2"] = pd.to_datetime(flattened_data["Placed in Service Date 2"], errors='coerce').dt.strftime('%m/%d/%Y')



In [8]:
# Combine address fields
address_columns = ["PROJECT NAME & ADDRESS", "PROJECT NAME & ADDRESS 2", "PROJECT NAME & ADDRESS 3", "PROJECT NAME & ADDRESS 4", "PROJECT NAME & ADDRESS 5"]

# Extracts addresses from projects
def extract_address(x):
    # Project 27 has three full addresses, so we use the last one listed
    if x.name == 27:
        addr = x[address_columns].dropna().values[-1]
        return addr
    # The last two lines of the address field contain the address split between two lines, except for project 27
    else:
        addr = x[address_columns].dropna().values[-2:]
        return addr[0] + ", " + addr[1]

# Extract an address for each project
flattened_data["address"] = flattened_data.apply(extract_address, axis=1)

In [9]:
# Print the data to a csv
flattened_data.to_csv("data/processed_data.csv", index=False)

In [10]:
# Print the addresses to a seperate list
flattened_data["address"].to_csv("data/DSHA_addresses.csv", index=False)

At this point we transfer the address csv over to the geolocator to get the latitude and longitude of each project

In [11]:
# Read the file of geolocated addresses
geolocations = pd.read_csv("data/counts_per_tract.csv").drop_duplicates("input addresses")
# Join the geolocations to the flattened dataframe
geolocated_data = flattened_data.merge(geolocations, left_on="address", right_on="input addresses", how="inner")
# Remove lat,lot from unsuccessfully (not in the u.s.) geolocated address
geolocated_data.loc[geolocated_data["census tract"] == "Unable To Geolocate The Address", "lot"] = np.nan
geolocated_data.loc[geolocated_data["census tract"] == "Unable To Geolocate The Address", "lat"] = np.nan

In [12]:
# Convert lat,lot to Shapely points
geolocated_data = gpd.GeoDataFrame(geolocated_data, geometry=gpd.points_from_xy(geolocated_data['lat'], geolocated_data['lot'], crs="EPSG:4326"))


In [13]:
# Visualize points on a map

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[39.74503, -75.57203], zoom_start=14, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)


# Add markers for each school
points=folium.features.GeoJson(
        geolocated_data.loc[geolocated_data["census tract"] != "Unable To Geolocate The Address"], # Full geopandas data
        control=False,
        marker = folium.CircleMarker(radius = 5, # Radius in metres
                           weight = 0, #outline weight
                           fill_color = '#d95f02', 
                           fill_opacity = 1)
        )

points.add_to(us_map)
us_map

In [14]:
# extract shape files for senate districts
senate_districts = gpd.read_file("data/Enacted-Senate-EsriShp (1).zip")

# Download area of land shapes
worldland = gpd.read_file("data/ne_10m_land.zip")
# Intersect senate districts with land area to trim water (Use higher resolution shapefiles to improve trim quality)
senate_districts["geometry"] = senate_districts["geometry"].apply(lambda x: worldland["geometry"].intersection(x))[0]

# Gets the senate district containing a point
def get_district(x):
    # Return a blank when an address could not be geolocated
    if x["census tract"] == "Unable To Geolocate The Address":
        return ""
    # Return the senate district containing the point otherwise
    else:
        return senate_districts.loc[x["geometry"].within(senate_districts["geometry"])]["DISTRICT"].values[0]

# Add a column for senate district
geolocated_data["Senate District"] = geolocated_data.apply(get_district, axis=1)

In [15]:
#geolocated_data.loc[geolocated_data["Senate District"] == "5"]
geolocated_data["Senate District"].unique()

array(['3', '6', '20', '1', '17', '13', '11', '18', '', '21', '2', '15',
       '10', '14', '19', '16'], dtype=object)

In [16]:
# Add funding source column
geolocated_data["Funding Source"] = "LIHTC"

In [17]:
senate_districts

,ID,DATA,DISTRICT,MEMBERS,LOCKED,NAME,ADJ_POPULA,ADJ_WHITE,ADJ_BLACK,ADJ_ASIAN,...,DEVIATION,F_DEVIATIO,F_ADJ_WHIT,F_ADJ_BLAC,F_ADJ_ASIA,F_ADJ_AMIN,F_ADJ_OTHE,DISTRICT_N,DISTRICT_L,geometry
0,1,14,21,1.0,NaN,NaN,49173,32924,8657,702,...,2049.0,0.043481,0.669554,0.176052,0.014276,0.006243,0.055742,21,21|4.35%,"POLYGON ((-75.71231 38.70575, -75.71226 38.705..."
1,2,7,15,1.0,NaN,NaN,47131,36027,5734,569,...,7.0,0.000149,0.764401,0.121661,0.012073,0.008020,0.019329,15,15|0.01%,"POLYGON ((-75.76002 39.29682, -75.75626 39.297..."
2,3,9,17,1.0,NaN,NaN,49015,20911,19797,1596,...,1891.0,0.040128,0.426625,0.403897,0.032561,0.006039,0.037029,17,17|4.01%,"POLYGON ((-75.57702 39.20892, -75.57698 39.208..."
3,4,2,10,1.0,NaN,NaN,47345,28026,10997,3725,...,221.0,0.004690,0.591953,0.232274,0.078678,0.001816,0.026972,10,10|0.47%,"POLYGON ((-75.63571 39.46174, -75.63567 39.461..."
4,5,6,14,1.0,NaN,NaN,49189,29869,13166,928,...,2065.0,0.043821,0.607229,0.267661,0.018866,0.003761,0.024497,14,14|4.38%,"POLYGON ((-75.56886 39.44035, -75.55276 39.425..."
5,6,4,12,1.0,NaN,NaN,47368,26073,13463,2908,...,244.0,0.005178,0.550435,0.284221,0.061392,0.002977,0.028247,12,12|0.52%,"MULTIPOLYGON (((-75.54133 39.50886, -75.53734 ..."
6,7,3,11,1.0,NaN,NaN,47647,21194,13946,5106,...,523.0,0.011098,0.444813,0.292694,0.107163,0.004323,0.067685,11,11|1.11%,"POLYGON ((-75.70783 39.60969, -75.70888 39.609..."
7,8,20,8,1.0,NaN,NaN,45633,32402,5460,3875,...,-1491.0,-0.031640,0.710056,0.119650,0.084917,0.001096,0.017728,8,8|-3.16%,"POLYGON ((-75.73363 39.65391, -75.73359 39.653..."
8,9,21,9,1.0,NaN,NaN,45828,27596,7448,3187,...,-1296.0,-0.027502,0.602165,0.162521,0.069543,0.005302,0.067011,9,9|-2.75%,"POLYGON ((-75.66211 39.65930, -75.66213 39.659..."
9,10,5,13,1.0,NaN,NaN,48294,18048,19838,2801,...,1170.0,0.024828,0.373711,0.410776,0.057999,0.004410,0.072618,13,13|2.48%,"POLYGON ((-75.57683 39.69569, -75.57703 39.695..."


In [18]:
geolocated_data.drop(columns=["input addresses", "census tract", "lot", "lat"]).to_file("data/DSHA_districted.geojson", driver="GeoJSON")

In [19]:
geolocated_data.drop(columns=["input addresses", "census tract", "lot", "lat"]).columns

Index(['PROJECT NAME & ADDRESS', 'ALLOCATION AMOUNT', 'ALLOCATION DATE',
       'Placed in Service Date', '4% or 9% Allocation',
       'Applicable Credit Rate', 'Status Active/Non', '# of Tax Credit Units',
       'Type of Property', 'County', 'Tax Credit Compliance Date',
       'Extended Use Period', 'Tax Allocation Year',
       'PROJECT NAME & ADDRESS 2', 'ALLOCATION AMOUNT 2',
       'Placed in Service Date 2', 'Applicable Credit Rate 2',
       'PROJECT NAME & ADDRESS 3', 'PROJECT NAME & ADDRESS 4',
       'PROJECT NAME & ADDRESS 5', 'address', 'geometry', 'Senate District',
       'Funding Source'],
      dtype='object')

In [20]:
geolocated_data['# of Tax Credit Units'] = geolocated_data['# of Tax Credit Units'].astype(int)
aggregated_data = geolocated_data.groupby("Senate District").sum()["# of Tax Credit Units"].reset_index()
aggregated_data.to_csv("data/Tax_Credit_Units_per_Senate_District.csv", index=False)





/var/folders/9r/738fljg527g64ktptsnnzlpw0000gn/T/ipykernel_2686/449460401.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = geolocated_data.groupby("Senate District").sum()["# of Tax Credit Units"].reset_index()


In [21]:
data_noTBD = geolocated_data.loc[geolocated_data['ALLOCATION AMOUNT'] != "TBD"]
data_noTBD["ALLOCATION AMOUNT"] = data_noTBD['ALLOCATION AMOUNT'].astype(int)
aggregated_data = aggregated_data.merge(data_noTBD.groupby("Senate District").sum()["ALLOCATION AMOUNT"].reset_index(), how="outer", on="Senate District")

/Users/emiller/opt/miniconda3/envs/DSHA/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/9r/738fljg527g64ktptsnnzlpw0000gn/T/ipykernel_2686/4095451458.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = aggregated_data.merge(data_noTBD.groupby("Senate District").sum()["ALLOCATION AMOUNT"].reset_index(), how="outer", on="Senate District")


In [22]:
# Tax Credit Units, Allocation Amount, Average Allocation per 100 persons, avg_population_per_tax_credit, adj_popula
aggregated_data = aggregated_data.merge(gpd.read_file("data/aggregated_senate_new.json")[["name", "district", "adj_popula"]], how="outer", left_on="Senate District", right_on="district").drop(columns="Senate District")
aggregated_data["Average Allocation per 100 Persons"] = aggregated_data["ALLOCATION AMOUNT"] * 100 / aggregated_data["adj_popula"].astype(float)
aggregated_data["Average Population per Tax Credit Unit"] = aggregated_data["adj_popula"].astype(float) / aggregated_data["# of Tax Credit Units"]
aggregated_data["Average Allocation per Tax Credit Unit"] = aggregated_data["ALLOCATION AMOUNT"].astype(float) / aggregated_data["# of Tax Credit Units"]
aggregated_data["Funding Source"] = "LIHTC"

# Convert adjusted population to int
aggregated_data["adj_popula"] = aggregated_data["adj_popula"].fillna(0).astype(int)


aggregated_data

,# of Tax Credit Units,ALLOCATION AMOUNT,name,district,adj_popula,Average Allocation per 100 Persons,Average Population per Tax Credit Unit,Average Allocation per Tax Credit Unit,Funding Source
0,70.0,1247683.0,NaN,NaN,0,NaN,NaN,17824.042857,LIHTC
1,260.0,2345086.0,Sarah Mcbride,1,44856,5228.031924,172.523077,9019.561538,LIHTC
2,106.0,656670.0,Stephanie L. Hansen,10,47281,1388.866564,446.047170,6195.000000,LIHTC
3,60.0,774946.0,Bryan Townsend,11,48203,1607.671722,803.383333,12915.766667,LIHTC
4,120.0,497801.0,Marie Pinkney,13,48294,1030.771939,402.450000,4148.341667,LIHTC
5,54.0,NaN,Kyra Hoffner,14,49253,NaN,912.092593,NaN,LIHTC
6,158.0,3013465.0,Dave Lawson,15,47104,6397.471552,298.126582,19072.563291,LIHTC
7,36.0,999013.0,Eric Buckson,16,47958,2083.099796,1332.166667,27750.361111,LIHTC
8,210.0,620003.0,Trey Paradee,17,49042,1264.228620,233.533333,2952.395238,LIHTC
9,102.0,1021863.0,Dave Wilson,18,48592,2102.944929,476.392157,10018.264706,LIHTC


In [23]:
long_data = pd.melt(aggregated_data, id_vars=["district", "name", "Funding Source"], value_vars=["# of Tax Credit Units", "ALLOCATION AMOUNT", "adj_popula", "Average Allocation per 100 Persons", "Average Population per Tax Credit Unit", "Average Allocation per Tax Credit Unit"])
averages = long_data.groupby("variable").mean(numeric_only=True).reset_index()
averages["Funding Source"] = "LIHTC"
averages["name"] = np.nan
averages["district"] = "District Average"
long_data = pd.concat([long_data,averages])
long_data.fillna(0).to_csv("long_tax_data.csv", index=False)

In [57]:
long_data

,district,name,Funding Source,variable,value
0,NaN,NaN,LIHTC,# of Tax Credit Units,70.0
1,1,Sarah Mcbride,LIHTC,# of Tax Credit Units,260.0
2,10,Stephanie L. Hansen,LIHTC,# of Tax Credit Units,106.0
3,11,Bryan Townsend,LIHTC,# of Tax Credit Units,60.0
4,13,Marie Pinkney,LIHTC,# of Tax Credit Units,120.0
...,...,...,...,...,...
105,7,Spiros Mantzavinos,LIHTC,Average Population per Tax Credit Unit,NaN
106,12,Nicole Poore,LIHTC,Average Population per Tax Credit Unit,NaN
107,9,Jack Walsh,LIHTC,Average Population per Tax Credit Unit,NaN
108,5,Kyle Evans Gay,LIHTC,Average Population per Tax Credit Unit,NaN


Figure out how senate districts overlap with senate districts

In [ ]:
# Download census block groups
blocks = requests.get("https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_10_bg_500k.zip")
blocks = gpd.read_file(BytesIO(blocks.content))
blocks

In [ ]:
lines = []

# creating a pdf file object
with open('data/CensusBlockBreakdownbySenateDistrict.pdf', 'rb') as pdfFileObj:
    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    # printing number of pages in pdf file
    print(pdfReader.numPages)

    # Iterate over each page
    for pageNum in range(pdfReader.numPages):
        # creating a page object
        pageObj = pdfReader.getPage(pageNum)
        
        # extracting text from page
        pageText = pageObj.extractText()
        
        # extract lines from each page
        pageLines = pageText.split("\n")
        
        for line in pageLines:
            lines.append(line.split(' '))

# Pull the columns out and separate columns that were incorrectly joined
columns = lines[0]
columns[1] = columns[0][5:] + columns[1]
columns[0] = columns[0][:5]
columns.insert(2, columns[2][:6])
columns[3] = columns[3][6:]
data = [x for x in lines if x[0].isdigit()]
for row in data:
    row.insert(1, row[0][2:])
    row[0] = row[0][:2]
    row.insert(2, row[2][:5])
    row[3] = row[3][5:]
    
senate_census_map = pd.DataFrame(data, columns=columns)
senate_census_map

In [ ]:
crosstab = pd.crosstab(senate_census_map["Proposed2022_SD"], senate_census_map["BlockGroup"])
senate_per_bg = crosstab.apply(lambda x : x[x != 0].index.values)
senate_per_bg.loc[senate_per_bg.apply(len) != 1]




In [ ]:
senate_per_bg2 = crosstab.apply(lambda x : x[x != 0].values)
senate_per_bg2.loc[senate_per_bg2.apply(len) != 1]

In [ ]:
scbg = pd.concat([senate_per_bg, senate_per_bg2], axis=1)
scbg.loc[(scbg.apply(lambda x : x.apply(len)) != 1).all(axis=1)].values

In [ ]:
scbg.columns = ["Senate Districts", "Blocks per District"]
scbg.to_csv("Senate_to_Block_Groups.csv")

In [22]:
# Download trimmed senate distrcit shapes
trim = gpd.read_file("data/2022Senate_Districts_Trimmed.geojson", driver="GeoJSON")
trim["district"] = trim["SLDUST"].astype(int)
trim = trim[["district", "geometry"]]

In [23]:
aggregated_data["district"] = pd.to_numeric(aggregated_data["district"], errors="coerce")
aggregated_data = aggregated_data.merge(trim, on="district", how="outer")
aggregated_data

,# of Tax Credit Units,ALLOCATION AMOUNT,name,district,adj_popula,Average Allocation per 100 Persons,Average Population per Tax Credit Unit,Average Allocation per Tax Credit Unit,Funding Source,geometry
0,70.0,1247683.0,NaN,NaN,NaN,NaN,NaN,17824.042857,LIHTC,None
1,260.0,2345086.0,Sarah Mcbride,1.0,44856,5228.031924,172.523077,9019.561538,LIHTC,"MULTIPOLYGON (((-75.58733 39.77320, -75.58637 ..."
2,106.0,656670.0,Stephanie L. Hansen,10.0,47281,1388.866564,446.047170,6195.000000,LIHTC,"MULTIPOLYGON (((-75.78504 39.60622, -75.78319 ..."
3,60.0,774946.0,Bryan Townsend,11.0,48203,1607.671722,803.383333,12915.766667,LIHTC,"MULTIPOLYGON (((-75.78754 39.63903, -75.78376 ..."
4,120.0,497801.0,Marie Pinkney,13.0,48294,1030.771939,402.450000,4148.341667,LIHTC,"MULTIPOLYGON (((-75.70558 39.61057, -75.70340 ..."
5,54.0,NaN,Kyra Hoffner,14.0,49253,NaN,912.092593,NaN,LIHTC,"MULTIPOLYGON (((-75.76798 39.39332, -75.76098 ..."
6,158.0,3013465.0,Dave Lawson,15.0,47104,6397.471552,298.126582,19072.563291,LIHTC,"MULTIPOLYGON (((-75.76010 39.29682, -75.75000 ..."
7,36.0,999013.0,Eric Buckson,16.0,47958,2083.099796,1332.166667,27750.361111,LIHTC,"MULTIPOLYGON (((-75.56902 39.00676, -75.56048 ..."
8,210.0,620003.0,Trey Paradee,17.0,49042,1264.228620,233.533333,2952.395238,LIHTC,"MULTIPOLYGON (((-75.60524 39.16461, -75.60423 ..."
9,102.0,1021863.0,Dave Wilson,18.0,48592,2102.944929,476.392157,10018.264706,LIHTC,"MULTIPOLYGON (((-75.72260 38.82986, -75.61542 ..."


In [34]:
gpd.GeoDataFrame(aggregated_data.loc[~aggregated_data["district"].isna()]).to_file("data/aggregated_with_geo.geojson", driver="GeoJSON")
aggregated_data.to_csv("data/aggregated_data_with_na.csv", index=False)

In [ ]:
# Plot the data on a heatmap

columns = list(np.setdiff1d(senate_districts.columns.values, ["geometry"]))

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[38.9108, -75.5277], zoom_start=8, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)

# Style and highlight functions map population values to color values
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':'red', 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

# Add a map over the tiles with the given colors and a tooltip
NIL=folium.features.GeoJson(
        senate_districts, # Full geopandas data
        style_function=style_function, # function for base colors
        control=False,
        highlight_function=highlight_function, # function for color upon mouse hover
        tooltip=folium.features.GeoJsonTooltip(fields=columns, # data to show in tooltip
            aliases=columns, # Names for tooltip data
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"), # coloring and font info for tooltip
            sticky=True
        )
    )

# Add elements to map
us_map.add_child(NIL)